In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score
from typing import List, Dict, Tuple
import torch.nn.functional as F
from sklearn.metrics.pairwise import rbf_kernel
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import multivariate_normal

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from warnings import filterwarnings
filterwarnings('ignore')

def mmd_loss(source_features: torch.Tensor, target_features: torch.Tensor,
             kernel_type: str = 'rbf',
             sigma: float = 1.0) -> torch.Tensor:
    def gaussian_kernel(x, y, sigma=1.0):
        x_size = x.size(0)
        y_size = y.size(0)

        # Compute squared pairwise distances
        norm2 = torch.pow(x, 2).sum(1).unsqueeze(1) - 2 * torch.mm(x, y.t()) + torch.pow(y, 2).sum(1).unsqueeze(0)

        # Compute kernel
        return torch.exp(-norm2 / (2 * sigma * sigma))

    def linear_kernel(x, y):
        return torch.mm(x, y.t())

    # Choose kernel
    if kernel_type == 'rbf':
        kernel = lambda x, y: gaussian_kernel(x, y, sigma)
    else:
        kernel = linear_kernel

    # Compute kernel matrices
    xx = kernel(source_features, source_features)
    yy = kernel(target_features, target_features)
    xy = kernel(source_features, target_features)

    # Compute MMD
    xx_mean = xx.mean()
    yy_mean = yy.mean()
    xy_mean = xy.mean()

    return xx_mean + yy_mean - 2 * xy_mean

def coral_loss(self, source, target):

    d = source.size(1)

    # Source covariance
    source = source - torch.mean(source, dim=0, keepdim=True)
    source_cov = torch.matmul(source.t(), source) / (source.size(0) - 1)


    target = target - torch.mean(target, dim=0, keepdim=True)
    target_cov = torch.matmul(target.t(), target) / (target.size(0) - 1)

    # Frobenius norm between the two covariance matrices
    loss = torch.norm(source_cov - target_cov, p='fro')
    return loss / (4 * d * d)

class ProbabilisticLwPClassifier:
    def __init__(self, uncertainty_threshold=0.7, feature_dim=512):
        self.class_prototypes = {}
        self.class_uncertainties = {}
        self.class_counts = {}
        self.uncertainty_threshold = uncertainty_threshold
        self.reg_matrix = torch.eye(feature_dim) * 1e-4
        self.feature_dim = feature_dim



    @classmethod
    def load_model(cls, filepath):
        drive_path = '/content/drive/MyDrive/ml_models/'
        full_filepath = os.path.join(drive_path, filepath)

        try:
            model_state = torch.load(full_filepath)

            loaded_model = cls(
                uncertainty_threshold=model_state['uncertainty_threshold'],
                feature_dim=model_state['feature_dim']
            )

            loaded_model.class_prototypes = {
                k: torch.tensor(v) for k, v in model_state['class_prototypes'].items()
            }

            loaded_model.class_uncertainties = {
                k: torch.tensor(v) for k, v in model_state['class_uncertainties'].items()
            }

            loaded_model.class_counts = model_state['class_counts']

            print(f"Model loaded successfully from {full_filepath}")
            return loaded_model

        except Exception as e:
            print(f"Error loading model: {e}")
            return None

    def train(self, train_data: torch.Tensor, train_labels: torch.Tensor,
              prev_prototypes=None, prev_uncertainties=None, prev_counts=None):
        unique_labels = torch.unique(train_labels)

        for label in unique_labels:
            mask = train_labels == label
            class_samples = train_data[mask]

            if len(class_samples) < 2:
                continue

            current_mean = torch.median(class_samples, dim=0)[0]
            centered_data = class_samples - current_mean

            current_cov = self._compute_robust_covariance(centered_data)
            current_count = len(class_samples)

            if prev_prototypes is not None and label.item() in prev_prototypes:
                prev_count = prev_counts[label.item()]
                total_count = current_count + prev_count
                adaptive_alpha = 0.125 * current_count / total_count

                previous_mean = prev_prototypes[label.item()]
                combined_mean = adaptive_alpha * current_mean + (1 - adaptive_alpha) * previous_mean

                previous_cov = prev_uncertainties[label.item()]
                combined_cov = adaptive_alpha * current_cov + (1 - adaptive_alpha) * previous_cov

                combined_count = current_count + prev_count
            else:
                combined_mean = current_mean
                combined_cov = current_cov
                combined_count = current_count

            self.class_prototypes[label.item()] = combined_mean
            self.class_uncertainties[label.item()] = combined_cov
            self.class_counts[label.item()] = combined_count

        return self.class_prototypes, self.class_uncertainties

    def _compute_robust_covariance(self, centered_data: torch.Tensor) -> torch.Tensor:
        n_samples = len(centered_data)

        sample_cov = torch.mm(centered_data.T, centered_data) / (n_samples - 1)

        distances = torch.sqrt(torch.sum(centered_data ** 2, dim=1))
        weights = 1 / (1 + distances)
        weighted_cov = (weights[:, None, None] * torch.bmm(centered_data[:, :, None], centered_data[:, None, :])).mean(0)

        alpha = min(0.5, 1.0 / n_samples)

        target = torch.diag(torch.diag(weighted_cov))

        cov = (1 - alpha) * weighted_cov + alpha * target + self.reg_matrix
        return cov

    def compute_probability_batch(self, samples: torch.Tensor) -> torch.Tensor:
        n_samples = len(samples)
        n_classes = max(self.class_prototypes.keys()) + 1
        log_probs = torch.full((n_samples, n_classes), float('-inf'), device=samples.device, dtype=samples.dtype)

        for label in self.class_prototypes:
            mean = self.class_prototypes[label]
            cov = self.class_uncertainties[label]

            try:
                cov_pinv = torch.linalg.pinv(cov)
                log_det = torch.logdet(cov + 1e-6 * torch.eye(cov.size(0), device=cov.device, dtype=cov.dtype))

                diff = samples - mean.unsqueeze(0)
                mahalanobis_dist = torch.einsum('bi,ij,bj->b', diff, cov_pinv, diff)

                entropy = torch.sum(-torch.diagonal(cov) * torch.log(torch.diagonal(cov) + 1e-6))
                total_entropy = sum(
                    -torch.sum(torch.diagonal(self.class_uncertainties[cls]) * torch.log(torch.diagonal(self.class_uncertainties[cls]) + 1e-6))
                    for cls in self.class_prototypes
                )
                class_prior = (total_entropy - entropy) / total_entropy

                log_prior = torch.log(torch.tensor(class_prior, device=samples.device, dtype=samples.dtype))

                log_probs[:, label] = -0.5 * (log_det + mahalanobis_dist + self.feature_dim * np.log(2 * np.pi)) + log_prior
            except Exception as e:
                print(f"Error processing label {label}: {e}")
                continue

        if torch.all(torch.isinf(log_probs)):
            return torch.ones((n_samples, n_classes), device=samples.device, dtype=samples.dtype) / n_classes

        log_probs = log_probs - torch.logsumexp(log_probs, dim=1, keepdim=True)
        return torch.exp(log_probs)

    def predict_proba(self, data: torch.Tensor) -> np.ndarray:
        with torch.no_grad():
            probs = self.compute_probability_batch(data)
            probs = torch.nan_to_num(probs, 0.0)
            probs = probs / torch.sum(probs, dim=1, keepdim=True)
            return probs.numpy()

    def predict(self, data: torch.Tensor) -> np.ndarray:
        probs = self.predict_proba(data)
        return np.argmax(probs, axis=1)

    def get_uncertainty(self, data: torch.Tensor) -> np.ndarray:
        probs = self.predict_proba(data)
        probs = np.clip(probs, 1e-10, 1.0)
        entropy = -np.sum(probs * np.log(probs), axis=1)
        max_entropy = -np.log(1.0 / len(self.class_prototypes))
        return entropy / max_entropy


class UDASequentialTrainer:
    def __init__(self, dataset_paths: list, heldout_paths: list,
                 mmd_weight: float = 0.1,
                 mmd_kernel: str = 'rbf'):
        self.datasets = dataset_paths
        self.mmd_weight = mmd_weight
        self.mmd_kernel = mmd_kernel

        self.heldout_datasets = heldout_paths
        self.models = []
        self.accuracies_matrix = np.zeros((10, 10))
        self.uncertainties_matrix = np.zeros((10, 10))

        self.feature_extractor = models.resnet34(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.feature_extractor.children())[:-1])
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.feature_extractor = self.feature_extractor.to(self.device)
        self.feature_extractor.eval()

        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )

        print(f"Using device: {self.device}")
    def compute_domain_adaptation_loss(self, source_features: torch.Tensor,
                                        target_features: torch.Tensor) -> torch.Tensor:
        return mmd_loss(source_features, target_features,
                        kernel_type=self.mmd_kernel)

    def preprocess_data(self, data):
        if data.max() > 1:
            data = data / 255.0
        if data.shape[1:] == (32, 32, 3):
            data = data.permute(0, 3, 1, 2)

        if self.feature_extractor.training:
            transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2)
            ])
            data = torch.stack([transform(img) for img in data])

        normalized_data = torch.stack([self.normalize(img) for img in data])
        return normalized_data

    def extract_features(self, data):
        normalized_data = self.preprocess_data(data)
        normalized_data = normalized_data.to(self.device)

        features_list = []
        batch_size = 64

        with torch.no_grad():
            for i in range(0, len(normalized_data), batch_size):
                batch = normalized_data[i:i + batch_size]
                features = self.feature_extractor(batch)
                features = features.view(features.size(0), -1)
                features = F.normalize(features, p=2, dim=1)
                features_list.append(features.cpu())

        return torch.cat(features_list, dim=0)

    def load_from_pth(self, path, is_eval=False):
        try:
            data_dict = torch.load(path)
            data = torch.tensor(data_dict['data'], dtype=torch.float32)
            targets = torch.tensor(data_dict['targets'], dtype=torch.long) if 'targets' in data_dict else None
            print(f"Loaded data from {path}: {data.shape}")
            return data, targets
        except Exception as e:
            print(f"Error loading data from {path}: {str(e)}")
            return None, None

    def train_model_on_d1(self):
        print("Training initial model on D1...")
        data, targets = self.load_from_pth(self.datasets[0])
        if data is None or targets is None:
            raise ValueError("Could not load initial training data")

        features = self.extract_features(data)
        print(f"Extracted features shape: {features.shape}")

        model = ProbabilisticLwPClassifier()
        model.train(features, targets)
        print(f"Created initial prototypes for {len(model.class_prototypes)} classes")
        self.models.append(model)

    def evaluate_model_on_data(self, model, data, targets):
        if data is None or targets is None:
            return 0.0, 1.0

        features = self.extract_features(data)
        predictions = model.predict(features)
        accuracy = accuracy_score(targets, predictions)
        uncertainties = model.get_uncertainty(features)
        mean_uncertainty = np.mean(uncertainties)

        return accuracy, mean_uncertainty



    def run_sequential_training(self, mmd_weight=0.1):
        print("Starting sequential probabilistic training with MMD adaptation...")

        if len(self.models) == 0:
            self.train_model_on_d1()
            self.evaluate_models()

        for i in tqdm(range(1, len(self.datasets)), desc="Training on sequential datasets"):
            print(f"\nProcessing dataset {i+1}")

            data, targets = self.load_from_pth(self.datasets[i])
            if data is None:
                continue

            current_features = self.extract_features(data)

            current_model = self.models[-1]
            previous_features = None

            if i > 0:
                prev_data, _ = self.load_from_pth(self.datasets[i-1])
                if prev_data is not None:
                    previous_features = self.extract_features(prev_data)

            pseudo_labels = current_model.predict(current_features)
            uncertainties = current_model.get_uncertainty(current_features)

            confidence_threshold = np.percentile(uncertainties, 70)
            confidence_mask = uncertainties < confidence_threshold

            confident_features = current_features[confidence_mask]
            confident_pseudo_labels = pseudo_labels[confidence_mask]

            if len(confident_features) < 100:
                print(f"Warning: Insufficient confident predictions ({len(confident_features)})")
                confident_features = current_features
                confident_pseudo_labels = pseudo_labels
            else:
                print(f"Using {len(confident_features)}/{len(current_features)} confident predictions")

            mmd_loss = 0.0
            if previous_features is not None:
                mmd_loss = mmd_loss(
                    source_features=previous_features,
                    target_features=current_features,
                    kernel_type='rbf'
                )
                print(f"MMD Domain Adaptation Loss: {mmd_loss.item()}")

            new_model = ProbabilisticLwPClassifier()
            new_model.train(
                confident_features,
                torch.tensor(confident_pseudo_labels),
                prev_prototypes=current_model.class_prototypes,
                prev_uncertainties=current_model.class_uncertainties,
                prev_counts=current_model.class_counts
            )

            if mmd_loss > 0:
                for label in new_model.class_prototypes:
                    adjustment = mmd_weight * mmd_loss.item()
                    new_model.class_prototypes[label] *= (1 - adjustment)

            self.models.append(new_model)

            self.evaluate_models()

        self.plot_metrics()

    def evaluate_models(self):
        print("\nEvaluating latest model...")
        current_model_index = len(self.models) - 1
        model = self.models[-1]

        for j in range(current_model_index + 1):
            data, targets = self.load_from_pth(self.heldout_datasets[j], is_eval=True)
            accuracy, uncertainty = self.evaluate_model_on_data(model, data, targets)
            self.accuracies_matrix[current_model_index, j] = accuracy
            self.uncertainties_matrix[current_model_index, j] = uncertainty
            print(f"Latest Model {current_model_index + 1} on Dataset {j + 1}:")
            print(f"  Accuracy: {accuracy:.4f}")
            print(f"  Uncertainty: {uncertainty:.4f}")

    def plot_metrics(self):
        plt.figure(figsize=(10, 8))
        sns.heatmap(self.accuracies_matrix, annot=True, fmt='.3f', cmap='YlOrRd')
        plt.title('Model Accuracy Matrix')
        plt.xlabel('Dataset Index')
        plt.ylabel('Model Index')
        plt.savefig('accuracy_matrix.png')
        plt.close()

        plt.figure(figsize=(10, 8))
        sns.heatmap(self.uncertainties_matrix, annot=True, fmt='.3f', cmap='YlOrRd')
        plt.title('Model Uncertainty Matrix')
        plt.xlabel('Dataset Index')
        plt.ylabel('Model Index')
        plt.savefig('uncertainty_matrix.png')
        plt.close()


class TaskTwoUDASequentialTrainer(UDASequentialTrainer):
    def __init__(self, dataset_paths: list, heldout_paths: list):
        super().__init__(dataset_paths, heldout_paths)

        self.accuracies_matrix = np.zeros((11, 21))
        self.uncertainties_matrix = np.zeros((11, 21))

    def run_sequential_training(self, initial_model):
        print("Starting sequential probabilistic training for Task 2...")

        self.models = [initial_model]
        self.evaluate_models()

        for i in tqdm(range(10, 20), desc="Training on sequential datasets"):
            print(f"\nProcessing dataset {i+1}")
            data, targets = self.load_from_pth(self.datasets[i])
            if data is None:
                continue

            features = self.extract_features(data)
            current_model = self.models[-1]

            pseudo_labels = current_model.predict(features)
            uncertainties = current_model.get_uncertainty(features)

            confidence_threshold = np.percentile(uncertainties, 75)
            confidence_mask = uncertainties < confidence_threshold
            confident_features = features[confidence_mask]
            confident_pseudo_labels = pseudo_labels[confidence_mask]

            if len(confident_features) < 50:
                print(f"Warning: Very few confident predictions ({len(confident_features)})")
                confident_features = features
                confident_pseudo_labels = pseudo_labels
            else:
                print(f"Using {len(confident_features)}/{len(features)} confident predictions")

            new_model = ProbabilisticLwPClassifier()
            new_model.train(
                confident_features,
                torch.tensor(confident_pseudo_labels),
                prev_prototypes=current_model.class_prototypes,
                prev_uncertainties=current_model.class_uncertainties,
                prev_counts=current_model.class_counts
            )
            self.models.append(new_model)
            self.evaluate_models()

        self.plot_metrics()

    def evaluate_models(self):
        print("\nEvaluating latest model...")
        current_model_index = len(self.models) - 1
        model = self.models[-1]

        for j in range(current_model_index + 10):
            if j < 10:
                data_path = f'/content/drive/MyDrive/dataset/part_one_dataset/eval_data/{j+1}_eval_data.tar.pth'
            else:
                data_path = f'/content/drive/MyDrive/dataset/part_two_dataset/eval_data/{j-9}_eval_data.tar.pth'

            data, targets = self.load_from_pth(data_path, is_eval=True)
            accuracy, uncertainty = self.evaluate_model_on_data(model, data, targets)
            self.accuracies_matrix[current_model_index, j] = accuracy
            self.uncertainties_matrix[current_model_index, j] = uncertainty

            dataset_desc = f"Part 1 Dataset {j+1}" if j < 10 else f"Part 2 Dataset {j-9}"
            print(f"Latest Model {current_model_index + 1} on {dataset_desc}:")
            print(f"  Accuracy: {accuracy:.4f}")
            print(f"  Uncertainty: {uncertainty:.4f}")

    def plot_metrics(self):
        max_used_col = 20
        filled_accuracies = self.accuracies_matrix[:, :max_used_col]
        filled_uncertainties = self.uncertainties_matrix[:, :max_used_col]

        plt.figure(figsize=(20, 8))
        x_labels = [f'Part 1 D{i+1}' for i in range(10)] + [f'Part 2 D{i+1}' for i in range(10)]
        sns.heatmap(filled_accuracies, annot=True, fmt='.3f', cmap='YlOrRd',
                    xticklabels=x_labels[:max_used_col])
        plt.title('Model Accuracy Matrix for Task 2')
        plt.xlabel('Dataset')
        plt.ylabel('Model Index')
        plt.tight_layout()
        plt.savefig('task2_accuracy_matrix.png')
        plt.close()

        plt.figure(figsize=(20, 8))
        sns.heatmap(filled_uncertainties, annot=True, fmt='.3f', cmap='YlOrRd',
                    xticklabels=x_labels[:max_used_col])
        plt.title('Model Uncertainty Matrix for Task 2')
        plt.xlabel('Dataset')
        plt.ylabel('Model Index')
        plt.tight_layout()
        plt.savefig('task2_uncertainty_matrix.png')
        plt.close()

def main():

    part_1_train_paths = [f'/content/drive/MyDrive/dataset/part_one_dataset/train_data/{i}_train_data.tar.pth' for i in range(1, 11)]
    part_1_eval_paths = [f'/content/drive/MyDrive/dataset/part_one_dataset/eval_data/{i}_eval_data.tar.pth' for i in range(1, 11)]

    part_2_train_paths = [f'/content/drive/MyDrive/dataset/part_two_dataset/train_data/{i}_train_data.tar.pth' for i in range(1, 11)]
    part_2_eval_paths = [f'/content/drive/MyDrive/dataset/part_two_dataset/eval_data/{i}_eval_data.tar.pth' for i in range(1, 11)]

    dataset_paths = part_1_train_paths + part_2_train_paths
    holdout_paths = part_1_eval_paths + part_2_eval_paths

    initial_model = ProbabilisticLwPClassifier.load_model('f10.pth')

    trainer = TaskTwoUDASequentialTrainer(dataset_paths, holdout_paths)

    trainer.run_sequential_training(initial_model)

if __name__ == "__main__":
    main()

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models
from sklearn.metrics import accuracy_score
from typing import List, Dict, Tuple
import torch.nn.functional as F
from sklearn.metrics.pairwise import rbf_kernel
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import multivariate_normal

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from warnings import filterwarnings
filterwarnings('ignore')

def mmd_loss(source_features: torch.Tensor, target_features: torch.Tensor,
             kernel_type: str = 'rbf',
             sigma: float = 1.0) -> torch.Tensor:
    def gaussian_kernel(x, y, sigma=1.0):
        x_size = x.size(0)
        y_size = y.size(0)

        # Compute squared pairwise distances
        norm2 = torch.pow(x, 2).sum(1).unsqueeze(1) - 2 * torch.mm(x, y.t()) + torch.pow(y, 2).sum(1).unsqueeze(0)

        # Compute kernel
        return torch.exp(-norm2 / (2 * sigma * sigma))

    def linear_kernel(x, y):
        return torch.mm(x, y.t())

    # Choose kernel
    if kernel_type == 'rbf':
        kernel = lambda x, y: gaussian_kernel(x, y, sigma)
    else:
        kernel = linear_kernel

    # Compute kernel matrices
    xx = kernel(source_features, source_features)
    yy = kernel(target_features, target_features)
    xy = kernel(source_features, target_features)

    # Compute MMD
    xx_mean = xx.mean()
    yy_mean = yy.mean()
    xy_mean = xy.mean()

    return xx_mean + yy_mean - 2 * xy_mean

def coral_loss(self, source, target):

    d = source.size(1)

    # Source covariance
    source = source - torch.mean(source, dim=0, keepdim=True)
    source_cov = torch.matmul(source.t(), source) / (source.size(0) - 1)


    target = target - torch.mean(target, dim=0, keepdim=True)
    target_cov = torch.matmul(target.t(), target) / (target.size(0) - 1)

    # Frobenius norm between the two covariance matrices
    loss = torch.norm(source_cov - target_cov, p='fro')
    return loss / (4 * d * d)

class ProbabilisticLwPClassifier:
    def __init__(self, uncertainty_threshold=0.7, feature_dim=512):
        self.class_prototypes = {}
        self.class_uncertainties = {}
        self.class_counts = {}
        self.uncertainty_threshold = uncertainty_threshold
        self.reg_matrix = torch.eye(feature_dim) * 1e-4
        self.feature_dim = feature_dim



    @classmethod
    def load_model(cls, filepath):
        drive_path = '/content/drive/MyDrive/ml_models/'
        full_filepath = os.path.join(drive_path, filepath)

        try:
            model_state = torch.load(full_filepath)

            loaded_model = cls(
                uncertainty_threshold=model_state['uncertainty_threshold'],
                feature_dim=model_state['feature_dim']
            )

            loaded_model.class_prototypes = {
                k: torch.tensor(v) for k, v in model_state['class_prototypes'].items()
            }

            loaded_model.class_uncertainties = {
                k: torch.tensor(v) for k, v in model_state['class_uncertainties'].items()
            }

            loaded_model.class_counts = model_state['class_counts']

            print(f"Model loaded successfully from {full_filepath}")
            return loaded_model

        except Exception as e:
            print(f"Error loading model: {e}")
            return None

    def train(self, train_data: torch.Tensor, train_labels: torch.Tensor,
              prev_prototypes=None, prev_uncertainties=None, prev_counts=None):
        unique_labels = torch.unique(train_labels)

        for label in unique_labels:
            mask = train_labels == label
            class_samples = train_data[mask]

            if len(class_samples) < 2:
                continue

            current_mean = torch.median(class_samples, dim=0)[0]
            centered_data = class_samples - current_mean

            current_cov = self._compute_robust_covariance(centered_data)
            current_count = len(class_samples)

            if prev_prototypes is not None and label.item() in prev_prototypes:
                prev_count = prev_counts[label.item()]
                total_count = current_count + prev_count
                adaptive_alpha = 0.125 * current_count / total_count

                previous_mean = prev_prototypes[label.item()]
                combined_mean = adaptive_alpha * current_mean + (1 - adaptive_alpha) * previous_mean

                previous_cov = prev_uncertainties[label.item()]
                combined_cov = adaptive_alpha * current_cov + (1 - adaptive_alpha) * previous_cov

                combined_count = current_count + prev_count
            else:
                combined_mean = current_mean
                combined_cov = current_cov
                combined_count = current_count

            self.class_prototypes[label.item()] = combined_mean
            self.class_uncertainties[label.item()] = combined_cov
            self.class_counts[label.item()] = combined_count

        return self.class_prototypes, self.class_uncertainties

    def _compute_robust_covariance(self, centered_data: torch.Tensor) -> torch.Tensor:
        n_samples = len(centered_data)

        sample_cov = torch.mm(centered_data.T, centered_data) / (n_samples - 1)

        distances = torch.sqrt(torch.sum(centered_data ** 2, dim=1))
        weights = 1 / (1 + distances)
        weighted_cov = (weights[:, None, None] * torch.bmm(centered_data[:, :, None], centered_data[:, None, :])).mean(0)

        alpha = min(0.5, 1.0 / n_samples)

        target = torch.diag(torch.diag(weighted_cov))

        cov = (1 - alpha) * weighted_cov + alpha * target + self.reg_matrix
        return cov

    def compute_probability_batch(self, samples: torch.Tensor) -> torch.Tensor:
        n_samples = len(samples)
        n_classes = max(self.class_prototypes.keys()) + 1
        log_probs = torch.full((n_samples, n_classes), float('-inf'), device=samples.device, dtype=samples.dtype)

        for label in self.class_prototypes:
            mean = self.class_prototypes[label]
            cov = self.class_uncertainties[label]

            try:
                cov_pinv = torch.linalg.pinv(cov)
                log_det = torch.logdet(cov + 1e-6 * torch.eye(cov.size(0), device=cov.device, dtype=cov.dtype))

                diff = samples - mean.unsqueeze(0)
                mahalanobis_dist = torch.einsum('bi,ij,bj->b', diff, cov_pinv, diff)

                entropy = torch.sum(-torch.diagonal(cov) * torch.log(torch.diagonal(cov) + 1e-6))
                total_entropy = sum(
                    -torch.sum(torch.diagonal(self.class_uncertainties[cls]) * torch.log(torch.diagonal(self.class_uncertainties[cls]) + 1e-6))
                    for cls in self.class_prototypes
                )
                class_prior = (total_entropy - entropy) / total_entropy

                log_prior = torch.log(torch.tensor(class_prior, device=samples.device, dtype=samples.dtype))

                log_probs[:, label] = -0.5 * (log_det + mahalanobis_dist + self.feature_dim * np.log(2 * np.pi)) + log_prior
            except Exception as e:
                print(f"Error processing label {label}: {e}")
                continue

        if torch.all(torch.isinf(log_probs)):
            return torch.ones((n_samples, n_classes), device=samples.device, dtype=samples.dtype) / n_classes

        log_probs = log_probs - torch.logsumexp(log_probs, dim=1, keepdim=True)
        return torch.exp(log_probs)

    def predict_proba(self, data: torch.Tensor) -> np.ndarray:
        with torch.no_grad():
            probs = self.compute_probability_batch(data)
            probs = torch.nan_to_num(probs, 0.0)
            probs = probs / torch.sum(probs, dim=1, keepdim=True)
            return probs.numpy()

    def predict(self, data: torch.Tensor) -> np.ndarray:
        probs = self.predict_proba(data)
        return np.argmax(probs, axis=1)

    def get_uncertainty(self, data: torch.Tensor) -> np.ndarray:
        probs = self.predict_proba(data)
        probs = np.clip(probs, 1e-10, 1.0)
        entropy = -np.sum(probs * np.log(probs), axis=1)
        max_entropy = -np.log(1.0 / len(self.class_prototypes))
        return entropy / max_entropy


class UDASequentialTrainer:
    def __init__(self, dataset_paths: list, heldout_paths: list,
                 mmd_weight: float = 0.1,
                 mmd_kernel: str = 'rbf'):
        self.datasets = dataset_paths
        self.mmd_weight = mmd_weight
        self.mmd_kernel = mmd_kernel

        self.heldout_datasets = heldout_paths
        self.models = []
        self.accuracies_matrix = np.zeros((10, 10))
        self.uncertainties_matrix = np.zeros((10, 10))

        self.feature_extractor = models.resnet34(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.feature_extractor.children())[:-1])
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.feature_extractor = self.feature_extractor.to(self.device)
        self.feature_extractor.eval()

        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )

        print(f"Using device: {self.device}")
    def compute_domain_adaptation_loss(self, source_features: torch.Tensor,
                                        target_features: torch.Tensor) -> torch.Tensor:
        return mmd_loss(source_features, target_features,
                        kernel_type=self.mmd_kernel)

    def preprocess_data(self, data):
        if data.max() > 1:
            data = data / 255.0
        if data.shape[1:] == (32, 32, 3):
            data = data.permute(0, 3, 1, 2)

        if self.feature_extractor.training:
            transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                transforms.ColorJitter(brightness=0.2, contrast=0.2)
            ])
            data = torch.stack([transform(img) for img in data])

        normalized_data = torch.stack([self.normalize(img) for img in data])
        return normalized_data

    def extract_features(self, data):
        normalized_data = self.preprocess_data(data)
        normalized_data = normalized_data.to(self.device)

        features_list = []
        batch_size = 64

        with torch.no_grad():
            for i in range(0, len(normalized_data), batch_size):
                batch = normalized_data[i:i + batch_size]
                features = self.feature_extractor(batch)
                features = features.view(features.size(0), -1)
                features = F.normalize(features, p=2, dim=1)
                features_list.append(features.cpu())

        return torch.cat(features_list, dim=0)

    def load_from_pth(self, path, is_eval=False):
        try:
            data_dict = torch.load(path)
            data = torch.tensor(data_dict['data'], dtype=torch.float32)
            targets = torch.tensor(data_dict['targets'], dtype=torch.long) if 'targets' in data_dict else None
            print(f"Loaded data from {path}: {data.shape}")
            return data, targets
        except Exception as e:
            print(f"Error loading data from {path}: {str(e)}")
            return None, None

    def train_model_on_d1(self):
        print("Training initial model on D1...")
        data, targets = self.load_from_pth(self.datasets[0])
        if data is None or targets is None:
            raise ValueError("Could not load initial training data")

        features = self.extract_features(data)
        print(f"Extracted features shape: {features.shape}")

        model = ProbabilisticLwPClassifier()
        model.train(features, targets)
        print(f"Created initial prototypes for {len(model.class_prototypes)} classes")
        self.models.append(model)

    def evaluate_model_on_data(self, model, data, targets):
        if data is None or targets is None:
            return 0.0, 1.0

        features = self.extract_features(data)
        predictions = model.predict(features)
        accuracy = accuracy_score(targets, predictions)
        uncertainties = model.get_uncertainty(features)
        mean_uncertainty = np.mean(uncertainties)

        return accuracy, mean_uncertainty



    def run_sequential_training(self, mmd_weight=0.1):
        print("Starting sequential probabilistic training with MMD adaptation...")

        if len(self.models) == 0:
            self.train_model_on_d1()
            self.evaluate_models()

        for i in tqdm(range(1, len(self.datasets)), desc="Training on sequential datasets"):
            print(f"\nProcessing dataset {i+1}")

            data, targets = self.load_from_pth(self.datasets[i])
            if data is None:
                continue

            current_features = self.extract_features(data)

            current_model = self.models[-1]
            previous_features = None

            if i > 0:
                prev_data, _ = self.load_from_pth(self.datasets[i-1])
                if prev_data is not None:
                    previous_features = self.extract_features(prev_data)

            pseudo_labels = current_model.predict(current_features)
            uncertainties = current_model.get_uncertainty(current_features)

            confidence_threshold = np.percentile(uncertainties, 70)
            confidence_mask = uncertainties < confidence_threshold

            confident_features = current_features[confidence_mask]
            confident_pseudo_labels = pseudo_labels[confidence_mask]

            if len(confident_features) < 100:
                print(f"Warning: Insufficient confident predictions ({len(confident_features)})")
                confident_features = current_features
                confident_pseudo_labels = pseudo_labels
            else:
                print(f"Using {len(confident_features)}/{len(current_features)} confident predictions")

            mmd_loss = 0.0
            if previous_features is not None:
                mmd_loss = mmd_loss(
                    source_features=previous_features,
                    target_features=current_features,
                    kernel_type='rbf'
                )
                print(f"MMD Domain Adaptation Loss: {mmd_loss.item()}")

            new_model = ProbabilisticLwPClassifier()
            new_model.train(
                confident_features,
                torch.tensor(confident_pseudo_labels),
                prev_prototypes=current_model.class_prototypes,
                prev_uncertainties=current_model.class_uncertainties,
                prev_counts=current_model.class_counts
            )

            if mmd_loss > 0:
                for label in new_model.class_prototypes:
                    adjustment = mmd_weight * mmd_loss.item()
                    new_model.class_prototypes[label] *= (1 - adjustment)

            self.models.append(new_model)

            self.evaluate_models()

        self.plot_metrics()

    def evaluate_models(self):
        print("\nEvaluating latest model...")
        current_model_index = len(self.models) - 1
        model = self.models[-1]

        for j in range(current_model_index + 1):
            data, targets = self.load_from_pth(self.heldout_datasets[j], is_eval=True)
            accuracy, uncertainty = self.evaluate_model_on_data(model, data, targets)
            self.accuracies_matrix[current_model_index, j] = accuracy
            self.uncertainties_matrix[current_model_index, j] = uncertainty
            print(f"Latest Model {current_model_index + 1} on Dataset {j + 1}:")
            print(f"  Accuracy: {accuracy:.4f}")
            print(f"  Uncertainty: {uncertainty:.4f}")

    def plot_metrics(self):
        plt.figure(figsize=(10, 8))
        sns.heatmap(self.accuracies_matrix, annot=True, fmt='.3f', cmap='YlOrRd')
        plt.title('Model Accuracy Matrix')
        plt.xlabel('Dataset Index')
        plt.ylabel('Model Index')
        plt.savefig('accuracy_matrix.png')
        plt.close()

        plt.figure(figsize=(10, 8))
        sns.heatmap(self.uncertainties_matrix, annot=True, fmt='.3f', cmap='YlOrRd')
        plt.title('Model Uncertainty Matrix')
        plt.xlabel('Dataset Index')
        plt.ylabel('Model Index')
        plt.savefig('uncertainty_matrix.png')
        plt.close()


class TaskTwoUDASequentialTrainer(UDASequentialTrainer):
    def __init__(self, dataset_paths: list, heldout_paths: list):
        super().__init__(dataset_paths, heldout_paths)

        self.accuracies_matrix = np.zeros((11, 21))
        self.uncertainties_matrix = np.zeros((11, 21))

    def run_sequential_training(self, initial_model):
        print("Starting sequential probabilistic training for Task 2...")

        self.models = [initial_model]
        self.evaluate_models()

        for i in tqdm(range(10, 20), desc="Training on sequential datasets"):
            print(f"\nProcessing dataset {i+1}")
            data, targets = self.load_from_pth(self.datasets[i])
            if data is None:
                continue

            features = self.extract_features(data)
            current_model = self.models[-1]

            pseudo_labels = current_model.predict(features)
            uncertainties = current_model.get_uncertainty(features)

            confidence_threshold = np.percentile(uncertainties, 75)
            confidence_mask = uncertainties < confidence_threshold
            confident_features = features[confidence_mask]
            confident_pseudo_labels = pseudo_labels[confidence_mask]

            if len(confident_features) < 50:
                print(f"Warning: Very few confident predictions ({len(confident_features)})")
                confident_features = features
                confident_pseudo_labels = pseudo_labels
            else:
                print(f"Using {len(confident_features)}/{len(features)} confident predictions")

            new_model = ProbabilisticLwPClassifier()
            new_model.train(
                confident_features,
                torch.tensor(confident_pseudo_labels),
                prev_prototypes=current_model.class_prototypes,
                prev_uncertainties=current_model.class_uncertainties,
                prev_counts=current_model.class_counts
            )
            self.models.append(new_model)
            self.evaluate_models()

        self.plot_metrics()

    def evaluate_models(self):
        print("\nEvaluating latest model...")
        current_model_index = len(self.models) - 1
        model = self.models[-1]

        for j in range(current_model_index + 10):
            if j < 10:
                data_path = f'/content/drive/MyDrive/dataset/part_one_dataset/eval_data/{j+1}_eval_data.tar.pth'
            else:
                data_path = f'/content/drive/MyDrive/dataset/part_two_dataset/eval_data/{j-9}_eval_data.tar.pth'

            data, targets = self.load_from_pth(data_path, is_eval=True)
            accuracy, uncertainty = self.evaluate_model_on_data(model, data, targets)
            self.accuracies_matrix[current_model_index, j] = accuracy
            self.uncertainties_matrix[current_model_index, j] = uncertainty

            dataset_desc = f"Part 1 Dataset {j+1}" if j < 10 else f"Part 2 Dataset {j-9}"
            print(f"Latest Model {current_model_index + 1} on {dataset_desc}:")
            print(f"  Accuracy: {accuracy:.4f}")
            print(f"  Uncertainty: {uncertainty:.4f}")

    def plot_metrics(self):
        max_used_col = 20
        filled_accuracies = self.accuracies_matrix[:, :max_used_col]
        filled_uncertainties = self.uncertainties_matrix[:, :max_used_col]

        plt.figure(figsize=(20, 8))
        x_labels = [f'Part 1 D{i+1}' for i in range(10)] + [f'Part 2 D{i+1}' for i in range(10)]
        sns.heatmap(filled_accuracies, annot=True, fmt='.3f', cmap='YlOrRd',
                    xticklabels=x_labels[:max_used_col])
        plt.title('Model Accuracy Matrix for Task 2')
        plt.xlabel('Dataset')
        plt.ylabel('Model Index')
        plt.tight_layout()
        plt.savefig('task2_accuracy_matrix.png')
        plt.close()

        plt.figure(figsize=(20, 8))
        sns.heatmap(filled_uncertainties, annot=True, fmt='.3f', cmap='YlOrRd',
                    xticklabels=x_labels[:max_used_col])
        plt.title('Model Uncertainty Matrix for Task 2')
        plt.xlabel('Dataset')
        plt.ylabel('Model Index')
        plt.tight_layout()
        plt.savefig('task2_uncertainty_matrix.png')
        plt.close()

def main():

    part_1_train_paths = [f'/content/drive/MyDrive/dataset/part_one_dataset/train_data/{i}_train_data.tar.pth' for i in range(1, 11)]
    part_1_eval_paths = [f'/content/drive/MyDrive/dataset/part_one_dataset/eval_data/{i}_eval_data.tar.pth' for i in range(1, 11)]

    part_2_train_paths = [f'/content/drive/MyDrive/dataset/part_two_dataset/train_data/{i}_train_data.tar.pth' for i in range(1, 11)]
    part_2_eval_paths = [f'/content/drive/MyDrive/dataset/part_two_dataset/eval_data/{i}_eval_data.tar.pth' for i in range(1, 11)]

    dataset_paths = part_1_train_paths + part_2_train_paths
    holdout_paths = part_1_eval_paths + part_2_eval_paths

    initial_model = ProbabilisticLwPClassifier.load_model('f10.pth')

    trainer = TaskTwoUDASequentialTrainer(dataset_paths, holdout_paths)

    trainer.run_sequential_training(initial_model)

if __name__ == "__main__":
    main()

Mounted at /content/drive
Model loaded successfully from /content/drive/MyDrive/ml_models/f10.pth


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 134MB/s]


Using device: cpu
Starting sequential probabilistic training for Task 2...

Evaluating latest model...
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/1_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 1 on Part 1 Dataset 1:
  Accuracy: 0.5936
  Uncertainty: 0.0128
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/2_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 1 on Part 1 Dataset 2:
  Accuracy: 0.5888
  Uncertainty: 0.0135
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/3_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 1 on Part 1 Dataset 3:
  Accuracy: 0.5816
  Uncertainty: 0.0115
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/4_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 1 on Part 1 Dataset 4:
  Accuracy: 0.5828
  Uncertainty: 0.0141
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/5_eval_dat

Training on sequential datasets:   0%|          | 0/10 [00:00<?, ?it/s]


Processing dataset 11
Loaded data from /content/drive/MyDrive/dataset/part_two_dataset/train_data/1_train_data.tar.pth: torch.Size([2500, 32, 32, 3])
Using 1875/2500 confident predictions

Evaluating latest model...
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/1_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 2 on Part 1 Dataset 1:
  Accuracy: 0.5928
  Uncertainty: 0.0133
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/2_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 2 on Part 1 Dataset 2:
  Accuracy: 0.5884
  Uncertainty: 0.0134
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/3_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 2 on Part 1 Dataset 3:
  Accuracy: 0.5808
  Uncertainty: 0.0117
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/4_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 2 on Part 1 Dataset 4:
  Accuracy: 

...

Latest Model 9 on Part 2 Dataset 8:
  Accuracy: 0.4680
  Uncertainty: 0.0139

Processing dataset 19
Loaded data from /content/drive/MyDrive/dataset/part_two_dataset/train_data/9_train_data.tar.pth: torch.Size([2500, 32, 32, 3])
Using 1875/2500 confident predictions

Evaluating latest model...
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/1_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 10 on Part 1 Dataset 1:
  Accuracy: 0.5924
  Uncertainty: 0.0130
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/2_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 10 on Part 1 Dataset 2:
  Accuracy: 0.5832
  Uncertainty: 0.0131
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/3_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 10 on Part 1 Dataset 3:
  Accuracy: 0.5768
  Uncertainty: 0.0117
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/4_eval_data.tar.pth

Training on sequential datasets:  90%|█████████ | 9/10 [33:26<04:09, 249.79s/it]

Latest Model 10 on Part 2 Dataset 9:
  Accuracy: 0.5096
  Uncertainty: 0.0124

Processing dataset 20
Loaded data from /content/drive/MyDrive/dataset/part_two_dataset/train_data/10_train_data.tar.pth: torch.Size([2500, 32, 32, 3])
Using 1875/2500 confident predictions

Evaluating latest model...
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/1_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 11 on Part 1 Dataset 1:
  Accuracy: 0.5920
  Uncertainty: 0.0133
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/2_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 11 on Part 1 Dataset 2:
  Accuracy: 0.5836
  Uncertainty: 0.0128
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/3_eval_data.tar.pth: torch.Size([2500, 32, 32, 3])
Latest Model 11 on Part 1 Dataset 3:
  Accuracy: 0.5768
  Uncertainty: 0.0118
Loaded data from /content/drive/MyDrive/dataset/part_one_dataset/eval_data/4_eval_data.tar.p

Training on sequential datasets: 100%|██████████| 10/10 [38:12<00:00, 229.30s/it]

Latest Model 11 on Part 2 Dataset 10:
  Accuracy: 0.5284
  Uncertainty: 0.0134
